## __Exploratory Data Analysis__

### __Import Packages__

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

from library.sb_utils import save_file

### __Load Data__

In [111]:
# Load cleaned data
df = pd.read_csv('./data/df_cleaned.csv')

# Load dictionaries from dict_file.txt and replace string keys with int values
with open('./data/dict_file.txt', 'r') as file:
    for line in file:
        split = line.split(' ', maxsplit=2)
        globals()[split[0]] = eval(split[2])
        globals()[split[0]] = {int(k):v for k, v in globals()[split[0]].items()}

# Test that we have the dictionaries loaded correctly
print(mothers_occupation)

{0: 'Student', 1: 'Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers', 2: 'Specialists in Intellectual and Scientific Activities', 3: 'Intermediate Level Technicians and Professions', 4: 'Administrative staff', 5: 'Personal Services, Security and Safety Workers and Sellers', 6: 'Farmers and Skilled Workers in Agriculture, Fisheries and Forestry', 7: 'Skilled Workers in Industry, Construction and Craftsmen', 8: 'Installation and Machine Operators and Assembly Workers', 9: 'Unskilled Workers', 10: 'Armed Forces Professions', 90: 'Other Situation', 99: '(blank)', 122: 'Health professionals', 123: 'teachers', 125: 'Specialists in information and communication technologies (ICT)', 131: 'Intermediate level science and engineering technicians and professions', 132: 'Technicians and professionals, of intermediate level of health', 134: 'Intermediate level technicians from legal, social, sports, cultural and similar services', 141: 'Offic

### __Explore Data__

In [113]:
print('Total number of students: ', df.shape[0])
print('Most international students are: ', nationality[df.Nationality.value_counts().index[1]])

Total number of students:  4424
Most international students are:  Brazilian
